In [7]:
!pip install mlflow>=2.11.0
!pip install transformers peft accelerate bitsandbytes datasets -q -U
!pip install datasets
!pip install --upgrade git+https://github.com/huggingface/transformers
!pip install --upgrade torch torchvision
!pip install --upgrade accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tdntvbxi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tdntvbxi
  Resolved https://github.com/huggingface/transformers to commit a5e5c92aea1e99cb84d7342bd63826ca6cd884c4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import pandas as pd
from datasets import load_dataset
from IPython.display import HTML, display

dataset_name = "Asis41/SQL_TST"
dataset = load_dataset(dataset_name, split="train")


def display_table(dataset_or_sample):
    # A helper fuction to display a Transformer dataset or single sample contains multi-line string nicely
    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_rows", None)

    if isinstance(dataset_or_sample, dict):
        df = pd.DataFrame(dataset_or_sample, index=[0])
    else:
        df = pd.DataFrame(dataset_or_sample)

    html = df.to_html().replace("\\n", "<br>")
    styled_html = f"""<style> .dataframe th, .dataframe tbody td {{ text-align: left; padding-right: 30px; }} </style> {html}"""
    display(HTML(styled_html))


display_table(dataset.select(range(3)))

,Question,Answer,SQL,Context
0,Dame todas las materias impartidas por el profesor José Roderick Mireles Moreno.,SELECT MATERIA FROM escuela WHERE PROFESOR = 'JOSE RODERICK MIRELES MORENO',José Roderick Mireles Moreno imparte las materias de,"CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))"
1,Dame la hora y el profesor del grupo A,"SELECT hora, profesor FROM escuela WHERE GRUPO = 'A'",El grupo A1 tiene las clases y el profesor :,"CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))"
2,Dime todos los profesores que imparten la materia Mecatrónica,SELECT PROFESOR FROM escuela WHERE MATERIA = 'MECATRÓNICA',La materia Mecatrónica es impartida por los maestros,"CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))"


In [9]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Training dataset contains {len(train_dataset)} text-to-SQL pairs")
print(f"Test dataset contains {len(test_dataset)} text-to-SQL pairs")

Training dataset contains 64 text-to-SQL pairs
Test dataset contains 17 text-to-SQL pairs


In [10]:
PROMPT_TEMPLATE = """You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
{context}

### Question:
{question}

### Response:
{output}"""


def apply_prompt_template(row):
    prompt = PROMPT_TEMPLATE.format(
        question=row["Question"],
        context=row["Context"],
        output=row["Answer"],
    )
    return {"prompt": prompt}


train_dataset = train_dataset.map(apply_prompt_template)
display_table(train_dataset.select(range(1)))

,Question,Answer,SQL,Context,prompt
0,Dame la hora a la que materia Investigación en Ciencia y Tecnología se imparte en el grupo B,SELECT hora FROM escuela WHERE MATERIA = 'INVESTIGACIÓN EN CIENCIA Y TECNOLOGÍA' AND GRUPO = 'B',La materia Investigación en Ciencia y Tecnología se imparte en el grupo B a las :,"CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))","You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.### Table:CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))### Question:Dame la hora a la que materia Investigación en Ciencia y Tecnología se imparte en el grupo B### Response:SELECT hora FROM escuela WHERE MATERIA = 'INVESTIGACIÓN EN CIENCIA Y TECNOLOGÍA' AND GRUPO = 'B'"


In [11]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"

# You can use a different max length if your custom dataset has shorter/longer input sequences.
MAX_LENGTH = 256

quantization_config = BitsAndBytesConfig(
    # Load the model with 4-bit quantization
    load_in_4bit=True,
    # Use double quantization
    bnb_4bit_use_double_quant=True,
    # Use 4-bit Normal Float for storing the base model weights in GPU memory
    bnb_4bit_quant_type="nf4",
    # De-quantize the weights to 16-bit (Brain) float before the forward/backward pass
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=quantization_config)


tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=MAX_LENGTH,
    padding_side="left",
    add_eos_token=True,
)

  


def tokenize_and_pad_to_fixed_length(sample):
    result = tokenizer(
        sample["prompt"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  

tokenized_train_dataset = train_dataset.map(tokenize_and_pad_to_fixed_length)

assert all(len(x["input_ids"]) == MAX_LENGTH for x in tokenized_train_dataset)

display_table(tokenized_train_dataset.select(range(1)))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

,Question,Answer,SQL,Context,prompt,input_ids,attention_mask,labels
0,Dame la hora a la que materia Investigación en Ciencia y Tecnología se imparte en el grupo B,SELECT hora FROM escuela WHERE MATERIA = 'INVESTIGACIÓN EN CIENCIA Y TECNOLOGÍA' AND GRUPO = 'B',La materia Investigación en Ciencia y Tecnología se imparte en el grupo B a las :,"CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))","You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.### Table:CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))### Question:Dame la hora a la que materia Investigación en Ciencia y Tecnología se imparte en el grupo B### Response:SELECT hora FROM escuela WHERE MATERIA = 'INVESTIGACIÓN EN CIENCIA Y TECNOLOGÍA' AND GRUPO = 'B'","[1, 995, 460, 264, 6787, 2245, 28733, 532, 28733, 9295, 2229, 28723, 12628, 272, 13208, 9882, 304, 4229, 3842, 2996, 28725, 574, 2389, 349, 298, 3324, 13208, 5709, 369, 11194, 272, 2996, 28723, 13, 13, 27332, 7582, 28747, 13, 21005, 23740, 3431, 13482, 325, 13, 2287, 334, 28779, 962, 8671, 1574, 920, 896, 13289, 28725, 13, 2287, 320, 1990, 4032, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, 2287, 4519, 28743, 1087, 28754, 725, 2377, 28754, 28779, 3402, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, 2287, 334, 1087, 28754, 725, 28741, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[1, 995, 460, 264, 6787, 2245, 28733, 532, 28733, 9295, 2229, 28723, 12628, 272, 13208, 9882, 304, 4229, 3842, 2996, 28725, 574, 2389, 349, 298, 3324, 13208, 5709, 369, 11194, 272, 2996, 28723, 13, 13, 27332, 7582, 28747, 13, 21005, 23740, 3431, 13482, 325, 13, 2287, 334, 28779, 962, 8671, 1574, 920, 896, 13289, 28725, 13, 2287, 320, 1990, 4032, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, 2287, 4519, 28743, 1087, 28754, 725, 2377, 28754, 28779, 3402, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, 2287, 334, 1087, 28754, 725, 28741, 27913, 12064, 28732, 28750, 28782, 28782, 557, 13, ...]"


In [12]:
import transformers

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipeline = transformers.pipeline(model=model, tokenizer=tokenizer, task="text-generation")

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer)) 

sample = test_dataset[1]
prompt = PROMPT_TEMPLATE.format(
    context=sample["Context"], question=sample["Question"], output=""
)  # Leave the answer part blank

with torch.no_grad():
    response = pipeline(prompt, max_new_tokens=256, repetition_penalty=1.15, return_full_text=False)

display_table({"prompt": prompt, "generated_query": response[0]["generated_text"]})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


,prompt,generated_query
0,"You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.### Table:CREATE TABLE escuela ( CUATRIMESTRE INT, TURNO VARCHAR(255), IDCARRERAGRUPO VARCHAR(255), CARRERA VARCHAR(255), NO INT, PROFESOR VARCHAR(255), MATERIA VARCHAR(255), escuela VARCHAR(255), DIA VARCHAR(255), CANTIDAD INT, GRUPO VARCHAR(255), GRUPO_EN_SISTEMA VARCHAR(255), AULA VARCHAR(255))### Question:¿Qué materias se imparten los sábados en el aula del auditorio?### Response:",SELECT * FROM escuela WHERE DIA = 'SAB' AND AULA = 'AUDI'


In [13]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Enabling gradient checkpointing, to make the training further efficient
model.gradient_checkpointing_enable()
# Set up the model for quantization-aware training e.g. casting layers, parameter freezing, etc.
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    # This is the rank of the decomposed matrices A and B to be learned during fine-tuning. A smaller number will save more GPU memory but might result in worse performance.
    r=32,
    # This is the coefficient for the learned ΔW factor, so the larger number will typically result in a larger behavior change after fine-tuning.
    lora_alpha=64,
    # Drop out ratio for the layers in LoRA adaptors A and B.
    lora_dropout=0.1,
    # We fine-tune all linear layers in the model. It might sound a bit large, but the trainable adapter size is still only **1.16%** of the whole model.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    # Bias parameters to train. 'none' is recommended to keep the original model performing equally when turning off the adapter.
    bias="none",
)

peft_model = get_peft_model(model, peft_config)

In [15]:
from datetime import datetime

import transformers
from transformers import TrainingArguments

#import mlflow

# Comment-out this line if you are running the tutorial on Databricks
#mlflow.set_experiment("MLflow PEFT Tutorial")

training_args = TrainingArguments(
    # Set this to mlflow for logging your training
    #report_to="mlflow",
    # Name the MLflow run
    run_name=f"Mistral-7B-SQL-QLoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
    # Replace with your output destination
    output_dir="TEST_1",
    # For the following arguments, refer to https://huggingface.co/docs/transformers/main_classes/trainer
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    bf16=False,
    learning_rate=2e-5,
    lr_scheduler_type="constant",
    max_steps=100,
    save_steps=100,
    logging_steps=100,
    warmup_steps=5,
    # https://discuss.huggingface.co/t/training-llama-with-lora-on-multiple-gpus-may-exist-bug/47005/3
    ddp_find_unused_parameters=False,
)

trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=tokenized_train_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=training_args,
)

# use_cache=True is incompatible with gradient checkpointing.
peft_model.config.use_cache = False

max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,0.160900


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=100, training_loss=0.16088205337524414, metrics={'train_runtime': 2219.1187, 'train_samples_per_second': 0.361, 'train_steps_per_second': 0.045, 'total_flos': 8842082766028800.0, 'train_loss': 0.16088205337524414, 'epoch': 12.5})

In [17]:
from huggingface_hub import login
login()

In [18]:
# push to the hub
model.push_to_hub("somosnlp/LLM_SQL_BaseDatosEspanol")# ,config=training_args)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/somosnlp/LLM_SQL_BaseDatosEspanol/commit/91874e83b97af64f30ecb40db3a5e76923b87aad', commit_message='Upload MistralForCausalLM', commit_description='', oid='91874e83b97af64f30ecb40db3a5e76923b87aad', pr_url=None, pr_revision=None, pr_num=None)